### Table of contests
1. [Data cleaning](#Data_cleaning)
2. [GPT-2 models](#GPT_2_models)
3. [Challenges and reflections](#Challenges_and_reflections)

### <a id="Data_cleaning">Data cleaning</a>

In [3]:
#imports
import os
import nltk
import re, string
import treetaggerwrapper
import json

In [4]:
#upload data
all_texts = []
names = os.listdir("/Users/valeriia/Desktop/archive")
for name in names:
    if name.endswith(".txt"):
        text = open("/Users/valeriia/Desktop/archive/"+name).read()
        all_texts.append(text)

In [5]:
#remove special characters before tokenization
def remove_characters_before_tokenization(sentence):
    sentence = sentence.strip()
    characters = r"[.|,|:|!|?|*|(|)|~|-|–|„|“|●|#|/|\"|»|«]"
    sentence = re.sub(characters, r"", sentence)
    repeat = r"[x+0-9]"
    sentence = re.sub(repeat, r"", sentence)
    if "&" in sentence:
        sentence = re.sub("&", r" and ", sentence)
    if "%" in sentence:
        sentence = re.sub("%", r" Prozent", sentence)
    if "-" in sentence:
        sentence = re.sub("-", r" ", sentence)
    if "'" in sentence:
        sentence = re.sub("'", r"", sentence)
    if "\n" in sentence:
        sentence = re.sub("\n", r" ### ", sentence) #mark spaces 
    return sentence

all_texts_cleaned = [remove_characters_before_tokenization(sentence) for sentence in all_texts]

In [6]:
#convert all words except nouns to lower case
tagger = treetaggerwrapper.TreeTagger(TAGLANG='de', TAGDIR="/Users/valeriia")
all_texts_cleaned2 = []
for text in all_texts_cleaned:
    text_tokens = []
    text_pos = tagger.tag_text(text)
    text_pos2 = treetaggerwrapper.make_tags(text_pos)
    for word in text_pos2:
        try:
            if word.pos == "NN" or word.pos == "NE":
                text_tokens.append(word.word)
            else:
                text_tokens.append((word.word).lower())
        except:
            continue
    text_cleaned = " ".join(text_tokens)
    all_texts_cleaned2.append(text_cleaned)

In [7]:
#put spaces "/n" back so that they can be used for marking borders between lines and refrains
all_texts_cleaned3 = []
def spaces(text):
    if "###" in text:
        text = re.sub(" ### ", r"\n", text)
        text = re.sub("### ", r"\n", text)
        text = re.sub("###", r"\n", text)
    all_texts_cleaned3.append(text)
    
for text in all_texts_cleaned2:
    spaces(text)

In [9]:
#save texts to a json file - each song is a separate string
jsonString = json.dumps(all_texts_cleaned3, indent=4, ensure_ascii=False)
jsonFile = open("all_texts_cleaned.json", "w", encoding='utf-8')
jsonFile.write(jsonString)
jsonFile.close()

In [10]:
#split the set into refrains
refrains = []
for text in all_texts_cleaned3:
    refrain = ""
    lines = text.splitlines( )
    for line in lines:
        refrain += line+"/n"
        if line == "" and refrain != "/n":
            if refrain.startswith("/n"):
                refrain = refrain[2:]
            refrains.append(refrain)
            refrain = ""

In [11]:
#save texts to a json file - each refrain is a separate string
jsonString = json.dumps(refrains, indent=4, ensure_ascii=False)
jsonFile = open("refrains.json", "w", encoding='utf-8')
jsonFile.write(jsonString)
jsonFile.close()

### <a id="GPT_2_models">GPT-2 models</a>

<a href="https://huggingface.co/dbmdz/german-gpt2" target="_blank">German GPT-2 model</a> was finetuned with <a href="https://www.kaggle.com/efrodl/german-rap-dataset" target="_blank">this rap dataset</a>. <a href="https://www.philschmid.de/fine-tune-a-non-english-gpt-2-model-with-huggingface" target="_blank">The following tutorial</a> was used as a basis for finetuning. For one model, the set was divided into separate songs; for another model - into separate refrains.

In [12]:
from transformers import pipeline

#### Examples for the first model

In [15]:
lyrics1 = pipeline("text-generation",model="gpt2_rap_lyrics_song_level",tokenizer="anonymous-german-nlp/german-gpt2")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [19]:
result = lyrics1('heute')[0]['generated_text']
print(result)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


heute geht die Meuterei los mit jedem Track wir sind zu weit unterwegs aber doch wir treffen uns immer mit keiner kann hier im Internet weiterreden denn ihr seid alle zu weit gekommen so viel habt ihr uns doch nicht geschenkt doch das alles ist die Geschichte


In [21]:
result = lyrics1('dieser Beat')[0]['generated_text']
print(result)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


dieser Beat ist mein Ding und deshalb machen sich noch mehr Fans die hier nichts zu sagen haben keine Mühe sondern rappen an die Grenzen der Zeit wenn mir mal jemand ne Waffe zeigt doch wer hat schon geahnt dass das anders war wie man die Dinge zu


In [26]:
result = lyrics1('Hassliebe ist')[0]['generated_text']
print(result)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Hassliebe ist nicht immer gut wenn ich die Augen zur Wand schließe Ich will nicht kratzen Ich hab’ keinen Bock drauf zu kratzen Ich sag’’ ich sag’ Ich hab’ kein Bock darauf zu kratzen Ich hab’ keinen Bock drauf


In [30]:
result = lyrics1('lass uns')[0]['generated_text']
print(result)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


lass uns ab jetzt wir sind am rumfahr n und wir mach keinen Schade wenn s dann passiert wo is er so wie ein Hustler in der Schweiz wo ist der Typ der uns im Ghetto antreibt und was ist es in deinen Augen so


#### Examples for the second model

In [17]:
lyrics2 = pipeline("text-generation",model="gpt2_rap_lyrics_refrain_level",tokenizer="anonymous-german-nlp/german-gpt2")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [20]:
result = lyrics2('heute')[0]['generated_text']
print(result)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


heute/n/n",  "Glaube ich will dich nicht mehr sehen/nIch hör die Stimme wie die Musik die mein Geist hörte/n/n",  "ihr kommt ja ich weiß was zu tun ist/nwas hat


In [25]:
result = lyrics2('dieser Beat')[0]['generated_text']
print(result)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


dieser Beat/nSido und der Junge aus dem Haus der Nasty Banger/n/n",  "denn sie haben schon im Schrank geschlafen dann sind es noch zwei Stunden bis die Erde schäumt/nIch fliege zu meiner


In [27]:
result = lyrics2('Hassliebe ist')[0]['generated_text']
print(result)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Hassliebe ist die Lüge die Lüge/nbist alle Menschen das ist Hass ist die Lüge/nder eine ist der der sich niemals fürchtet/ndie den Hass im Arm festhält/nAlles was ihn trifft wenn er auf Toilette wartet ist Hass


In [29]:
result = lyrics2('lass uns')[0]['generated_text']
print(result)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


lass uns noch zwei Wochen ins Hotel/nIch bleibe in meiner Welt mein Leben/ndann bleib ich dein Leben/nIch hab mein Geld in meine Hände gebracht/nIch war stolz auf Dich/nund ich habe den ganzen Tag gezaubert


### <a id="Challenges_and_reflections">Challenges and reflections</a>

<li>Out of two models, the second one which is trained on the refrain-level generates text which is subjectively closer to actual rap lyrics. Still, generated lyrics lack the right verse-form which is probably connected to the formatting of the data for finetuning.</li>
<li>Since there is no strict verse structure, it's not possible to measure the rhyme.</li>
<li>The dataset for training consisted of approx. 600 songs (50000 lines of lyrics). Ideally, the dataset should have been extended through additional lyrics which could be gathered by a lyrics scraper.</li>